In [2]:
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import MinMaxScaler

In [3]:
gpus = tf.config.list_physical_devices(device_type = 'GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

In [9]:
class DataLoader:
    def __init__(self):
        AS_dataset = pd.read_csv('./Arbitrary_Single_band_Coupler_Phase_Shift.csv', encoding='utf-8')
        self.X = AS_dataset.loc[:,'freq':'L4'].to_numpy()
        self.y = AS_dataset.loc[:,'S11r':'S41i'].to_numpy()
#         self.mmX = MinMaxScaler()
#         self.X[:,1:] = self.mmX.fit_transform(self.X[:,1:])
#         self.X[:,0] = self.X[:,0] / 10
#         self.X, _, self.y, _ = train_test_split(self.X, self.y, test_size=0.75, random_state=0)
        self.X_train, self.X_vali, self.y_train, self.y_vali = train_test_split(self.X, self.y, test_size=0.1, random_state=0)
        self.num_train = self.X_train.shape[0]
    def get_batch(self, batch_size=0, mode='train'):
        if mode == 'train':
            index = np.random.randint(0, self.num_train, batch_size)
            return self.X_train[index], self.y_train[index]
        if mode == 'validate':
            return self.X_vali, self.y_vali

In [10]:
class MLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.dense1 = tf.keras.layers.Dense(units=512, activation=tf.nn.leaky_relu)
        self.dense2 = tf.keras.layers.Dense(units=1024, activation=tf.nn.leaky_relu)
        self.dense3 = tf.keras.layers.Dense(units=512, activation=tf.nn.leaky_relu)
        self.dense4 = tf.keras.layers.Dense(units=256, activation=tf.nn.leaky_relu)
        self.dense5 = tf.keras.layers.Dense(units=8)
    
#     @tf.function
    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dense2(x)
        x = self.dense3(x)
        x = self.dense4(x)
        output = self.dense5(x)
        return output

In [11]:
num_epochs = 200
batch_size = 1024
learning_rate = 0.001

In [12]:
model = MLP()
data_loader = DataLoader()
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
X_v, y_v = data_loader.get_batch(mode='validate')
def train():
    num_batch = data_loader.num_train // batch_size
    for epoch_index in range(num_epochs):
        for batch in range(num_batch):
            X, y = data_loader.get_batch(batch_size)
            with tf.GradientTape() as tape:
                y_pred = model(X)
                tr_mse = tf.reduce_mean(tf.square(y_pred - y))
            grads = tape.gradient(tr_mse, model.variables)
            optimizer.apply_gradients(grads_and_vars=zip(grads, model.variables))
        if epoch_index % 10 == 0 or epoch_index == num_epochs - 1:
            tr_rmse = tf.sqrt(tr_mse)
            tr_mae = tf.reduce_mean(tf.abs(y_pred - y))
            tr_r2 = 1 - tf.reduce_sum(tf.square(y_pred - y)) / tf.reduce_sum(tf.square(y - tf.cast(tf.reduce_mean(y), dtype=tf.float32)))
            print("epoch:{}".format(epoch_index))
            print("train mse:{} rmse:{} mae:{} r2:{}".format(tr_mse, tr_rmse, tr_mae, tr_r2))
            y_v_p = model(X_v)
            va_mse = tf.reduce_mean(tf.square(y_v_p - y_v))
            va_rmse = tf.sqrt(va_mse)
            va_mae = tf.reduce_mean(tf.abs(y_v_p - y_v))
            va_r2 = 1 - tf.reduce_sum(tf.square(y_v_p - y_v)) / tf.reduce_sum(tf.square(y_v - tf.cast(tf.reduce_mean(y_v), dtype=tf.float32)))
            print("vali mse:{} rmse:{} mae:{} r2:{}".format(va_mse, va_rmse, va_mae, va_r2))

In [13]:
train()

epoch:0
train mse:0.0952557697892189 rmse:0.30863532423973083 mae:0.25238871574401855 r2:0.21212852001190186
vali mse:0.09371825307607651 rmse:0.3061343729496002 mae:0.24976204335689545 r2:0.22309726476669312
epoch:10
train mse:0.07130761444568634 rmse:0.2670348584651947 mae:0.21519824862480164 r2:0.40854591131210327
vali mse:0.07203582674264908 rmse:0.26839491724967957 mae:0.21656323969364166 r2:0.4028396010398865
epoch:20
train mse:0.06365450471639633 rmse:0.2522984445095062 mae:0.20191535353660583 r2:0.47458916902542114
vali mse:0.06593257933855057 rmse:0.2567733824253082 mae:0.2049683779478073 r2:0.4534341096878052
epoch:30
train mse:0.061959683895111084 rmse:0.24891701340675354 mae:0.19569209218025208 r2:0.4868156909942627
vali mse:0.05900618061423302 rmse:0.24291187524795532 mae:0.19354911148548126 r2:0.5108523368835449
epoch:40
train mse:0.05336909368634224 rmse:0.23101751506328583 mae:0.1824861466884613 r2:0.5586906671524048
vali mse:0.05311031639575958 rmse:0.23045675456523895

In [33]:
tf.saved_model.save(model, './models')

INFO:tensorflow:Assets written to: ./models\assets


In [14]:
model = tf.saved_model.load('./models')

In [42]:
def obj_func(s_para):
    E = tf.square(s_para)
    E11 = E[:,0] + E[:,1]
    E21 = E[:,2] + E[:,3]
    E31 = E[:,4] + E[:,5]
    E41 = E[:,6] + E[:,7]
    l1 = E11 - E21 - E31 + E41
    l2 = tf.square(E21 / (E31 + E21) - 2 / 3)
    l3 = tf.square(tf.math.atan2(s_para[:,5], s_para[:,4]) - tf.math.atan2(s_para[:,3], s_para[:,2]) - np.pi / 4)
    l4 = tf.square(tf.reduce_sum(E, axis=1) - 1)
    loss = l1 + l2 + l3 + l4
    return loss

In [43]:
num_nodes = 5000
num_node_epochs = 200

In [44]:
opt = tf.keras.optimizers.legacy.Adam(learning_rate=0.01)

mmin = np.min(data_loader.X[: , 1: ], axis=0)
mmax = np.max(data_loader.X[: , 1: ], axis=0)

# structure.append(tf.Variable(np.random.uniform(0, 1, (num_nodes, 10)), dtype=tf.float32))
structure = tf.Variable(np.random.uniform(mmin, mmax, (num_nodes, 12)), dtype=tf.float32)

freq1 = tf.ones([num_nodes, 1]) * 2.4
freq2 = tf.ones([num_nodes, 1]) * 2.5
freq3 = tf.ones([num_nodes, 1]) * 2.6

minLoss = 0
minIndex = 0
minS = 0

In [45]:
bestLoss = 10
bestStructure = 0

In [46]:
def check(structure):
    inva_place1 = tf.where(tf.logical_or(structure[:,:8] < 1, structure[:,:8] > 4))
    structure = tf.tensor_scatter_nd_update(structure, [inva_place1], [np.random.uniform(mmin[inva_place1[:,1]], mmax[inva_place1[:,1]], (inva_place1.shape[0]))])
    
    inva_place2 = tf.where(tf.logical_or(structure[:,8:] < 4, structure[:,8:] > 100)) + [0, 8]
    structure = tf.tensor_scatter_nd_update(structure, [inva_place2], [np.random.uniform(mmin[inva_place2[:,1]], mmax[inva_place2[:,1]], (inva_place2.shape[0]))])
    
    return tf.Variable(structure)
    
    # structure[j] = tf.Variable(tf.tensor_scatter_nd_update(structure[j], [nega_place], [np.random.uniform(0, 1, (nega_place.shape[0]))]))

In [13]:
def check(structure):
    inva_place1 = tf.where(tf.logical_or(structure[:,:8] < 0, structure[:,:8] > 10))
    structure = tf.tensor_scatter_nd_update(structure, [inva_place1], [np.random.uniform(mmin[inva_place1[:,1]], mmax[inva_place1[:,1]], (inva_place1.shape[0]))])
    
    inva_place2 = tf.where(tf.logical_or(structure[:,8:] < 1, structure[:,8:] > 100)) + [0, 8]
    structure = tf.tensor_scatter_nd_update(structure, [inva_place2], [np.random.uniform(mmin[inva_place2[:,1]], mmax[inva_place2[:,1]], (inva_place2.shape[0]))])
    
    inva_place3 = tf.where(structure[:,1] < structure[:,7]) # W2 < W8
    a = tf.concat([inva_place3, tf.ones([inva_place3.shape[0], 1], dtype=tf.int64)], axis=1)
    b = tf.concat([inva_place3, tf.ones([inva_place3.shape[0], 1], dtype=tf.int64) * 7], axis=1)
    ori = tf.concat([a, b], axis=0)
    cht = tf.concat([b, a], axis=0)
    structure = tf.tensor_scatter_nd_update(structure, [ori], [tf.gather_nd(structure, cht)])
    
    inva_place4 = tf.where(structure[:,1] < structure[:,0]) # W2 < W1
    a = tf.concat([inva_place4, tf.ones([inva_place4.shape[0], 1], dtype=tf.int64)], axis=1)
    b = tf.concat([inva_place4, tf.zeros([inva_place4.shape[0], 1], dtype=tf.int64)], axis=1)
    ori = tf.concat([a, b], axis=0)
    cht = tf.concat([b, a], axis=0)
    structure = tf.tensor_scatter_nd_update(structure, [ori], [tf.gather_nd(structure, cht)])
    
    inva_place5 = tf.where(structure[:,4] < structure[:,3]) # W5 < W4
    a = tf.concat([inva_place5, tf.ones([inva_place5.shape[0], 1], dtype=tf.int64) * 4], axis=1)
    b = tf.concat([inva_place5, tf.ones([inva_place5.shape[0], 1], dtype=tf.int64) * 3], axis=1)
    ori = tf.concat([a, b], axis=0)
    cht = tf.concat([b, a], axis=0)
    structure = tf.tensor_scatter_nd_update(structure, [ori], [tf.gather_nd(structure, cht)])
    
    inva_place6 = tf.where(structure[:,4] < structure[:,5]) # W5 < W6
    a = tf.concat([inva_place6, tf.ones([inva_place6.shape[0], 1], dtype=tf.int64) * 4], axis=1)
    b = tf.concat([inva_place6, tf.ones([inva_place6.shape[0], 1], dtype=tf.int64) * 5], axis=1)
    ori = tf.concat([a, b], axis=0)
    cht = tf.concat([b, a], axis=0)
    structure = tf.tensor_scatter_nd_update(structure, [ori], [tf.gather_nd(structure, cht)])
    
    return tf.Variable(structure)
    
    # structure[j] = tf.Variable(tf.tensor_scatter_nd_update(structure[j], [nega_place], [np.random.uniform(0, 1, (nega_place.shape[0]))]))

In [41]:
for i in range(num_node_epochs):
    with tf.GradientTape(watch_accessed_variables=False) as tape:
        tape.watch(structure)
        y_pred1 = model(tf.concat([freq1, structure], axis=1))
        y_pred2 = model(tf.concat([freq2, structure], axis=1))
        y_pred3 = model(tf.concat([freq3, structure], axis=1))
        loss = obj_func(y_pred1) + obj_func(y_pred2) + obj_func(y_pred3)
    minLoss = tf.reduce_min(loss).numpy()
    minIndex = tf.argmin(loss).numpy()
    minS = structure[minIndex].numpy()
    grads = tape.gradient(loss, structure)
    opt.apply_gradients(grads_and_vars=zip([grads], [structure]))
    structure = check(structure)
    if minLoss < bestLoss:
        bestLoss = minLoss
        bestStructure = minS
        # bestStructure = data_loader.mmX.inverse_transform([minS[0]])[0]
        print(minIndex)
        print(i, bestLoss)
        print(bestStructure)
        print()

-1.4841418
2702
0 -1.4841418
[ 1.205315   3.2317932  1.0367705  1.3162541  1.9051796  3.4026542
  1.0180898  4.46279   95.0443    61.16179   57.827282  21.524893 ]

-1.3353468
-1.3470559
-1.0160162
-1.0303379
-1.0438588
-1.0569081
-1.0695128
-1.08178
-1.0938578
-1.105901
-1.1179302
-1.1298566
-1.1417377
-1.1536366
-1.1654986
-1.1773059
-1.1889197
-1.2005298
-1.2121832
-1.2237055
-1.2351315
-1.2466139
-1.258227
-1.2699912
-1.2818677
-1.2939782
-1.306139
-1.3186135
-1.3313334
-1.3441751
-1.3571861
-1.3704666
-1.3837154
-1.397099
-1.410603
-1.4243273
-1.4382206
-1.4522957
-1.4667287
-1.1817831
-1.1960119
-1.2103151
-1.2247276
-1.1407487
-1.1507926
-1.1608348
-1.1709821
-1.1811669
-1.1915586
-1.2019892
-1.2188542
-1.2263175
-1.2369691
-1.2524076
-1.2680229
-1.2840406
-1.300142
-1.3167752
-1.3334457
-1.3493904
-1.365767
-1.382197
-1.3990701
-1.4161077
-1.4331266
-1.4500605
-1.4666924
-1.4835145
-1.5004816
3730
69 -1.5004816
[ 3.075873   3.0864484  1.4453377  2.2318661  2.5851016  3.7604563


-1.6924291
-1.7070934
-1.722347
-1.738199
-1.7537837
-1.7694862
-1.7847754
-1.7994347
-1.8137431
-1.827768
-1.8416548
-1.4713466
-1.4862618
-1.5011052
-1.5154402
-1.5300078
-1.543858
-1.5576881
-1.5713868
-1.5851058
-1.5988231
-1.6138616
-1.6275463
-1.6423978
-1.6573198
-1.6722163
-1.6869857
-1.7012517
-1.7149245
-1.7285869
-1.7410848
-1.7536918
-1.7657676
-1.7776015
-1.7894682
-1.8017185
-1.8139658
-1.8264031
-1.8390979
-1.8518898
-1.8649732
-1.878226
-1.891434
-1.9045186
-1.8533909
-1.8776517
-1.8991663
-1.836311
-1.8491632
-1.8615367
-1.8735429
-1.6819422
-1.7064071
-1.7308465
-1.7517006
-1.7726951
-1.7941824
-1.8162243
-1.8376403
-1.8590195
-1.8794905
-1.8989936
-1.918333
-1.9374418
-1.9564971
-1.9737813
-1.9901028
-2.0056787
-2.0211158
-2.035738
-2.0505414
-1.7305336
-1.7394214
-1.7482497
-1.75708
-1.7659535
-1.7746615
-1.7833548
-1.7923049
-1.8016782
-1.811122
-1.8208616
